# Classification
Learn a classifier and check how well it can predict the category and subcategory a person in your dataset belongs to.

In [1]:
# define a function which outputs accuracy, a confusion matrix, precision, recall and F1 for the results of your classification (when classifying into categories and when classifying into subcategories)

In [ ]:
# define a function which outputs a visualisation of the accuracy of your classifier per category and per subcategories